<a href="https://colab.research.google.com/github/PriyankaMittapelly/SaiPriyanka_INFO5731_Spring2020/blob/main/In_Class_exercises_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In class exercise 10 (20 points in total, 4/16/2021)

The purpose of the exercise is to practice different machine learning algorithms for text clustering
Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

Apply the listed clustering methods to the dataset:

K means, 
DBSCAN,
Hierarchical clustering. 

You can refer to of the codes from  the follwing link below. 
https://www.kaggle.com/karthik3890/text-clustering 
    

In [ ]:
import pandas as pd
import gensim
df=pd.read_csv("/content/Amazon_Unlocked_Mobile.csv")
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from textblob import Word
nltk.download('wordnet')
from nltk.stem import PorterStemmer
st = PorterStemmer()
df = df[df['Reviews'].notnull()]
stop = stopwords.words('english')
df['Punctuation'] = df['Reviews'].str.replace('[^\w\s].#','')
df['Removingstopwords'] =df['Punctuation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['Numerics']=df['Removingstopwords'].str.replace('[0-9]','')
df['lowercase'] =df['Numerics'].apply(lambda x: " ".join(x.lower() for x in x.split()))

df['Stemming']=df['lowercase'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
df['Final_text'] = df['Stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df['Final_text']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0         i feel lucki found use (phone u & use hard all...
1         nice phone, nice grade pantach revue. veri cle...
2                                                 veri plea
3           it work good goe slow sometim good phone i love
4         great phone replac lost phone. the thing volum...
                                ...                        
413835                         anoth great deal great price
413836                                                   ok
413837             pas everi drop test onto porcelain tile!
413838          i return meet need seem good select others.
413839    onli downsid appar verizon longer use v-cast m...
Name: Final_text, Length: 413778, dtype: object

In [ ]:
df_1=(df.sample(n=4000)).reset_index()
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(df_1['Final_text'].values)
tfidf.shape

from sklearn.cluster import KMeans
model_tf = KMeans(n_clusters = 5, n_jobs = -1,random_state=99)
model_tf.fit(tfidf)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=-1, precompute_distances='auto',
       random_state=99, tol=0.0001, verbose=0)

In [ ]:
labels_tf = model_tf.labels_
cluster_center_tf=model_tf.cluster_centers_
terms = tfidf_vect.get_feature_names()
terms[1:5]
df_2 = df_1
df_2['Tfidf Clus Label'] = model_tf.labels_
df_2

,index,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Punctuation,Removingstopwords,Numerics,lowercase,Stemming,Final_text,Tfidf Clus Label
0,99695,BlackBerry RCK71CW 9550 Unlocked Phone with Wi...,BlackBerry,29.99,1,i am disappointed in the seller of this phone ...,NaN,i am disappointed in the seller of this phone ...,disappointed seller phone purchase phone new p...,disappointed seller phone purchase phone new p...,disappointed seller phone purchase phone new p...,disappoint seller phone purchas phone new prob...,disappoint seller phone purchas phone new prob...,0
1,282580,Otterbox Defender Series Protective Case for S...,NaN,49.95,5,"Great product, highly recommend!!!!!",0.0,"Great product, highly recommend!!!!!","Great product, highly recommend!!!!!","Great product, highly recommend!!!!!","great product, highly recommend!!!!!","great product, highli recommend!!!!!","great product, highli recommend!!!!!",3
2,181555,FIGO Atrium 5.5 - Dual Micro SIM Unlocked 16GB...,FIGO,93.49,5,It worked fine for 2 days then it stopped read...,0.0,It worked fine for 2 days then it stopped read...,It worked fine 2 days stopped reading sims ......,It worked fine days stopped reading sims .......,it worked fine days stopped reading sims ....i...,it work fine day stop read sim ....i stop shop...,it work fine day stop read sim ....i stop shop...,0
3,387256,"Smart Watch,[U.S. Warranty]JoyGeek All-in-1 Bl...",JoyGeek,49.41,3,Does more then I thought. Bluetooth function w...,1.0,Does more then I thought. Bluetooth function w...,Does I thought. Bluetooth function works well....,Does I thought. Bluetooth function works well....,does i thought. bluetooth function works well....,doe i thought. bluetooth function work well. i...,doe i thought. bluetooth function work well. i...,0
4,221785,"LG G3 D855 Unlocked Cellphone, International V...",NaN,218.88,1,I bought the phone thinking it was original an...,2.0,I bought the phone thinking it was original an...,I bought phone thinking original began fail ne...,I bought phone thinking original began fail ne...,i bought phone thinking original began fail ne...,i bought phone think origin began fail next mo...,i bought phone think origin began fail next mo...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,108821,BLU Dash C Music-US GSM - Unlocked Cell Phone ...,BLU,54.99,4,excellent quality at a good price,0.0,excellent quality at a good price,excellent quality good price,excellent quality good price,excellent quality good price,excel qualiti good price,excel qualiti good price,4
3996,274546,Nokia Lumia 920 32GB Unlocked GSM Windows 8 Sm...,NaN,99.99,2,The Lumia 920 per se is a great phone. Althoug...,0.0,The Lumia 920 per se is a great phone. Althoug...,The Lumia 920 per se great phone. Although cou...,The Lumia per se great phone. Although course...,the lumia per se great phone. although course ...,the lumia per se great phone. although cours n...,the lumia per se great phone. although cours n...,0
3997,335489,Samsung Galaxy S Duos II S7582 DUAL SIM Factor...,Samsung,299.99,5,At first I did not like it because I did know ...,0.0,At first I did not like it because I did know ...,At first I like I know use it. Now I enjoying.,At first I like I know use it. Now I enjoying.,at first i like i know use it. now i enjoying.,at first i like i know use it. now i enjoying.,at first i like i know use it. now i enjoying.,0
3998,104382,BLU Advance 4.0L Unlocked Smartphone -Global G...,BLU,149.99,1,This was one of the worst products I ever boug...,3.0,This was one of the worst products I ever boug...,This one worst products I ever bought. Only tw...,This one worst products I ever bought. Only tw...,this one worst products i ever bought. only tw...,thi one worst product i ever bought. onli two ...,thi one worst product i ever bought. onli two ...,0


In [ ]:
df_2.groupby(['Tfidf Clus Label'])['Final_text'].count()

Tfidf Clus Label
0    3253
1     172
2     170
3     272
4     133
Name: Final_text, dtype: int64

In [ ]:
print("Top terms per cluster:")
order_centroids = model_tf.cluster_centers_.argsort()[:, ::-1]
for i in range(1,5):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :5]:
        print(' %s' % terms[ind], end='')
        print()

Top terms per cluster:
Cluster 1: good
 veri
 product
 phone
 it
Cluster 2: love
 it
 phone
 great
 new
Cluster 3: great
 excelent
 phone
 work
 product
Cluster 4: excel
 product
 thank
 phone
 price


In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
import matplotlib.pyplot as plt
import gensim
i=0
list_of_sent=[]
for sent in df_1['Final_text'].values:
    list_of_sent.append(sent.split())
w2v_model=gensim.models.Word2Vec(list_of_sent,size=100, workers=4)
import numpy as np
sent_vectors = [];
for sent in list_of_sent:
    sent_vec = np.zeros(100)
    cnt_words =0;
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
sent_vectors = np.array(sent_vectors)
sent_vectors = np.nan_to_num(sent_vectors)
sent_vectors.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


(4000, 100)

In [ ]:
minPts = 2 * 100
model = DBSCAN(eps = 8, min_samples = minPts, n_jobs=-2)
model.fit(sent_vectors)
df_1['AVG-W2V Clus Label'] = model.labels_
df_1.head(5)

,index,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Punctuation,Removingstopwords,Numerics,lowercase,Stemming,Final_text,Tfidf Clus Label,AVG-W2V Clus Label
0,99695,BlackBerry RCK71CW 9550 Unlocked Phone with Wi...,BlackBerry,29.99,1,i am disappointed in the seller of this phone ...,NaN,i am disappointed in the seller of this phone ...,disappointed seller phone purchase phone new p...,disappointed seller phone purchase phone new p...,disappointed seller phone purchase phone new p...,disappoint seller phone purchas phone new prob...,disappoint seller phone purchas phone new prob...,0,0
1,282580,Otterbox Defender Series Protective Case for S...,NaN,49.95,5,"Great product, highly recommend!!!!!",0.0,"Great product, highly recommend!!!!!","Great product, highly recommend!!!!!","Great product, highly recommend!!!!!","great product, highly recommend!!!!!","great product, highli recommend!!!!!","great product, highli recommend!!!!!",3,0
2,181555,FIGO Atrium 5.5 - Dual Micro SIM Unlocked 16GB...,FIGO,93.49,5,It worked fine for 2 days then it stopped read...,0.0,It worked fine for 2 days then it stopped read...,It worked fine 2 days stopped reading sims ......,It worked fine days stopped reading sims .......,it worked fine days stopped reading sims ....i...,it work fine day stop read sim ....i stop shop...,it work fine day stop read sim ....i stop shop...,0,0
3,387256,"Smart Watch,[U.S. Warranty]JoyGeek All-in-1 Bl...",JoyGeek,49.41,3,Does more then I thought. Bluetooth function w...,1.0,Does more then I thought. Bluetooth function w...,Does I thought. Bluetooth function works well....,Does I thought. Bluetooth function works well....,does i thought. bluetooth function works well....,doe i thought. bluetooth function work well. i...,doe i thought. bluetooth function work well. i...,0,0
4,221785,"LG G3 D855 Unlocked Cellphone, International V...",NaN,218.88,1,I bought the phone thinking it was original an...,2.0,I bought the phone thinking it was original an...,I bought phone thinking original began fail ne...,I bought phone thinking original began fail ne...,i bought phone thinking original began fail ne...,i bought phone think origin began fail next mo...,i bought phone think origin began fail next mo...,0,0


In [ ]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=7, affinity='euclidean', linkage='ward')  
Agg=cluster.fit_predict(sent_vectors)
df_1['AVG-W2V Clus Label'] = cluster.labels_
df_1.head(2)
df_1.groupby(['AVG-W2V Clus Label'])['Final_text'].count()
for i in range(5):
    print("Example for two reviews assigned to cluster ", i)
    print("-" * 70)
    print(df_1.iloc[df_1.groupby(['AVG-W2V Clus Label']).groups[i][0]]['Final_text'])
    print('\n')
    print(df_1.iloc[df_1.groupby(['AVG-W2V Clus Label']).groups[i][1]]['Final_text'])
    print('\n')
    print("_" * 70)

Example for two reviews assigned to cluster  0
----------------------------------------------------------------------
it work fine day stop read sim ....i stop shop amazon


great phone


______________________________________________________________________
Example for two reviews assigned to cluster  1
----------------------------------------------------------------------
ok


el producto fue recibido en excelent condicion parec nuevo se maneja muy bien con el sistema lo recomiendo ampliament


______________________________________________________________________
Example for two reviews assigned to cluster  2
----------------------------------------------------------------------
doe i thought. bluetooth function work well. it turn self self function work sim card fit. i believ watch made u.s. sim card.


i bought phone think origin began fail next month, work multitask key home , receiv call . i took servic told phone refurbish softwar problems, accept reinstal softwar


___________

In one paragraph, please compare K means, DBSCAN and Hierarchical clustering. 

In [ ]:
When you have unlabeled data, Kmeans clustering is used because it is a form of unsupervised learning. The aim of this algorithm is to find groups in the data, with K representing the number of groups.
DBSCAN is a clustering tool used in machine learning to distinguish high-density clusters from low-density clusters.
Hierarchical clustering, also known as hierarchical cluster analysis, is a method of grouping related objects into clusters.
Differences I noticed among them are that we know kmeans clusters unlabeled data, but when it comes to hirerarchical, 
it repeatedly links pairs of clusters until any data object is included in the hierarchy, while the relationship between clusters
in the kmeans algorithm is undetermined.Radius(R) and Minimum Points are the two parameters needed (M)and
R chooses a radius large enough to contain a dense area if there are enough points within it.
The minimum number of data points needed in a neighborhood to be classified as a cluster is determined by M.
The Kmeans clustering algorithm is unaffected by data point densities, and it only includes a certain number of clusters (K)


